In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_files = ["documents/Futbol.pdf"]  # Agrega la lista de archivos PDF

all_documents = []

for document in pdf_files:
    loader = PyPDFLoader(document) # Carga el documento
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)    # Divide el texto en chunks
    documents = text_splitter.split_documents(data)
    all_documents.extend(documents)

print(f"Se cargaron y dividieron {len(all_documents)} fragmentos de texto.")

Se cargaron y dividieron 7 fragmentos de texto.


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/rags/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_chroma import Chroma

# Crear un vector store de Chroma usando los documentos y el modelo de embeddings
vector_store = Chroma.from_documents(
    documents=all_documents,
    collection_name="pdfs_2",
    embedding=embeddings,  # Pasar el modelo de embeddings aquí
)

print(f"Se creó el vector store con {len(all_documents)} documentos.")

Se creó el vector store con 7 documentos.


In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdfs_2",
    embedding_function=embeddings,
)

In [6]:
retriever = vector_store.as_retriever()

In [7]:
question = "Como es la filmografia de Martin Scorsese"
docs = vector_store.similarity_search(question, k=10)
len(docs)

Number of requested results 10 is greater than number of elements in index 7, updating n_results = 7


7

In [8]:
docs[0]

Document(metadata={'page': 0, 'source': 'documents/Futbol.pdf'}, page_content='cabodel partido,deunaduraciónde90minutos,esel queresultaganadordelencuentro.')

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [10]:
chain.invoke("que es el futbol")

'Según el documento proporcionado, el fútbol (también conocido como football en algunos países) es un deporte equipo jugado entre dos conjuntos de once jugadores cada uno, con la objetivo de marcar más goles que el opposing equipo. Es considerado uno de los deportes más populares del mundo, practicados por más de 270 millones de personas. La versión moderna del juego se inventó en Inglaterra tras la formación de la Football Association, y las reglas actuales se basan en esas de 1863. La Federación Internacional de Fútbol Asociación (FIFA) es el organismo recto que rige el deporte internacional.'